# DC Motor Electromechanical Modeling

This *Jupyter Notebook* is dedicated to the study of the **Direct Current Motor Electrical and Mechanical Modeling**. 

The following study will be developed by the implementation of *Python* code for the application and visualization of the concepts learned with the use of *NumPy* and *Plotly* libraries. 

The following model is based and adapted from [this article](https://professorcesarcosta.com.br/upload/imagens_upload/Modelagem%20de%20um%20motor%20de%20corrente%20continua.pdf).

---

## The DC Motor

---

## Electrical Model of the Motor

The Kirchhoff's loop equations for the motor's armature is:

$$
v(t) = R_a \cdot i_a(t) + L_a \cdot \frac{d i_a(t)}{dt} + v_g(t)
$$

$v_g(t)$ is the back EMF voltage generated by the rotor's spinning motion, expressed as:

$$
v_g(t) = k \cdot \phi(t) \cdot \omega(t) 
$$

Where:
- $k$ is a magnetic coupling coefficient;
- $\Phi(t)$ is the magnetic flux density inside the motor;
- $\omega$ is the angular velocity of the rotor. 

Assuming a permanent magnet as the flux source and disregarding effects that may modify it such as armature reaction, $k$ and $\Phi(t)$ can be abstracted to a single constant $K_e$:

$$
v_g(t) = K_e \cdot \omega(t) 
$$

Then, the following expression can be achieved

$$
v(t) = R_a \cdot i_a(t) + L_a \cdot \frac{d i_a(t)}{dt} + K_e \cdot \omega(t) 
$$

Finishing the model as:

$$
\frac{d i_a(t)}{dt} = \frac{1}{L_a} \cdot v(t) - \frac{R_a}{L_a} \cdot i_a(t) - \frac{K_e}{L_a} \cdot \omega(t) 
$$

For a negligible motor inductance, the induced voltage at the armature $L_a \cdot \frac{d i_a(t)}{dt}$ can be diregarded and armature current can be directly obtained by:
$$
i_a(t) = \frac{v(t) - K_e \cdot \omega(t)}{R_a} 
$$

---

## Mechanical Model of the Motor

By Newton's second law of motion:

$$
\Tau_g = \Tau_f + \Tau_L + \Tau_J + \Tau_v
$$

Where:
- $\Tau_g$ is the torque generated by the motor;
- $\Tau_f$ is the torque needed to overcome friction losses;
- $\Tau_L$ is the load torque;
- $\Tau_J$ is the torque necessary to accelerate the inertia of the rotor;
- $\Tau_v$ is the viscous damping torque.

The inertial torque is dependant on the angular acceleration and the moment of inertia of the rotor:

$$
\Tau_J = J \cdot\frac{d\omega (t)}{dt}
$$

The vicous damping torque is directly proportional to the angular velocity of the rotor: 

$$
\Tau_v = b\cdot\omega (t)
$$

Thus, the model can be re-written as:

$$
\Tau_g = \Tau_f + \Tau_L + J\cdot\frac{d\omega (t)}{dt} + b\cdot\omega (t)
$$

For a free spinning motor with friction disregarded, the model may be simplified to:

$$
\Tau_g = J \cdot\frac{d\omega (t)}{dt} + b\cdot\omega (t)
$$

---

## The Electromechanical Model of the Motor

To link both electrical and mechanical models, it's possible to make a linear relation between the generated torque and the armature current flowing on the motor:

$$
T_g = K_t \cdot i_a(t)
$$

Where $K_t$ is the **torque constant** of the motor. By joining this equation with the mechanical model:

$$
K_t \cdot i_a(t) = J \cdot\frac{d\omega (t)}{dt} + b\cdot\omega (t)
$$

Which can be expressed as:

$$
\frac{d\omega (t)}{dt} = \frac{K_t}{J} \cdot i_a(t) - \frac{b}{J}\cdot\omega (t)
$$

For the simplified electrical model:

$$
K_t \cdot  \frac{v(t) - K_e \cdot \omega(t)}{R_a} = J \cdot\frac{d\omega (t)}{dt} + b\cdot\omega (t)
$$

Finally:

$$
\frac{d\omega (t)}{dt} = \frac{K_t}{R_a \cdot J} \cdot v(t) - \frac{K_t \cdot K_e + R_a \cdot b}{R_a \cdot J}\cdot\omega (t)
$$

---

In [31]:
# Importing modules...
import numpy as np

import sys
sys.path.append('..') # Go back to base directory

from modules.graph import *
from modules.viewer3d import *

In [32]:
# Direct current motor model
class DCMotor:
    def __init__(
        self,
        # Electrical parameters
        R_a, # Armature resistance
        L_a, # Armature inductance
        K_t, # Torque constant

        # Mechanical Parameters
        J, # Rotor's moment of inertia
        b, # Viscous damping coefficient

        dt=1e-4, # Differential time step

        # Initial motor status parameters (not spinning and no current flowing)
        w=0.0,  # Angular velocity
        i_a=0.0 # Armature current
    ):
        # Time parameters
        self.dt = dt

        # Electrical characteristics of the motor
        self.R_a = R_a
        self.L_a = L_a
        self.K_e = K_t
        self.K_t = K_t
 
        # Mechanical properties
        self.J = J
        self.b = b

        # Motor status
        self.i_a = i_a
        self.w = w
        
    def electromechanical_model(self, v, T):
        # Calculate and update state variables
        i_a_dot = (v - self.i_a * self.R_a - self.w * self.K_e) / self.L_a

        # Integrate using Euler's Method
        self.i_a += i_a_dot * self.dt

        # Calculate and update state variables
        w_dot = self.i_a * self.K_t / self.J - self.w * self.b / self.J - T / self.J
        
        # Integrate using Euler's Method
        self.w += w_dot * self.dt

    def get_output(self):
        return np.array(
            [
                [self.w], 
                [self.i_a]
            ]
        )

In [33]:
# Instanciating and simulating the model
DCM = DCMotor(
    # Electrical parameters
    R_a=3.0, # In ohms
    L_a=6.0e-3, # In henrys
    K_t=50.0e-3, # In N*m/A

    # Mechanical Parameters
    J=100e-6, # In Kg*m^2
    b=105e-6, # In Kg*m^2/s

    dt=1e-4, # Differential time step

    # Initial angular velocity
    w=0.0 # In rad/s
)

# Simulation time parameters
start_time = 0.0 # In Seconds
stop_time = 2.0  # In Seconds

time_vector = np.linspace(
    start_time, 
    stop_time, 
    int((stop_time - start_time) / DCM.dt)
)

# Generate PWM signal for motor's input
def pwm_signal(
    V_max, 
    duty_cycle, 
    frequency, 
    time_vector
):
    return V_max * (time_vector % (1 / frequency) < (1 / frequency) * duty_cycle / 100)

# PWM signal parameters 
V_cc = 10 # In volts
duty_cycle = 100 # In %
frequency = 490 # In Hz

v_signal = pwm_signal(V_cc, duty_cycle, frequency, time_vector)
T_signal = 0.05 * np.heaviside(time_vector - 1, 1) # Step function starting at 1 s

# Generate solution
output = []
for v, T, t in zip(v_signal, T_signal, time_vector):
    DCM.electromechanical_model(v, T)
    
    output.append(DCM.get_output())

output = np.hstack(output)

In [34]:
# Plot input variables
graph = Graph(
    title="Input in Time Domain", 
    axis_title=("Time (s)", "Armature Voltage (V)")
)

timespan = round(5/(frequency * DCM.dt)) # Numbers of samples for a timespan of 5 cycles

graph.add_trace(
    points=np.vstack((time_vector, v_signal))[:, :timespan], 
    name="v (V)"
)

graph.figure.show(renderer='notebook_connected')

In [35]:
# Plot input variables
graph = Graph(
    title="Output in Time Domain", 
    axis_title=("Time (s)", "Rotor Angular Velocity (rad/s)")
)

graph.add_trace(
    points=np.vstack((time_vector, output[0])), 
    name="w (rad/s)"
)

graph.figure.show(renderer='notebook_connected')

In [36]:
# Plot input variables
graph = Graph(
    title="Output in Time Domain", 
    axis_title=("Time (s)", "Armature Current (A)")
)

graph.add_trace(
    points=np.vstack((time_vector, output[1])), 
    name="i_a (A)"
)

graph.figure.show(renderer='notebook_connected')